In [ ]:
# 定义优化器
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=0.001)

# 训练过程中应用权重衰减
for inputs, labels in dataloader:
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)

    # 计算权重衰减项的梯度并添加到参数的梯度上
    for param in model.parameters():
        if param.requires_grad and param.grad is not None:
            param.grad.add(param.data, lambda=optimizer.param_groups[0]['weight_decay'])

    loss.backward()
    optimizer.step()

In [ ]:
import math
import numpy as np
import torch
import torch.nn as nn


class MyLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_features, out_features))
        self.bias = nn.Parameter(torch.randn(out_features))
        self.fc = nn.Linear(3, 3)

    def forward(self, input):
        return self.fc((input @ self.weight) + self.bias)
    
    
m = MyLinear(4, 3)
sample_input = torch.randn(4)
m(sample_input)
for name,m in m.named_parameters():
    print(name)
    

    loss.backward()
sample_input = torch.ones(4)*1
y = net(sample_input)
y.backward()

# 对模型的梯度进行裁剪
nn.utils.clip_grad_norm_(model.parameters(), max_norm)

optimizer.step()

In [42]:

class MyLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_features, out_features))
        self.bias = nn.Parameter(torch.randn(out_features))
        self.fc = nn.Linear(3, 3)

    def forward(self, input):
        return self.fc((input @ self.weight) + self.bias)
    
# using modules to building blocks
net = nn.Sequential()
net.add_module('1', MyLinear(4, 3))
net.add_module('3', nn.Linear(3, 1))

In [68]:
sample_input = torch.ones(4)*1
y = net(sample_input)
y.backward()
for m in net.parameters():
    print(m.grad)

tensor([[-1.5650, -1.0272, -0.4034],
        [-1.5650, -1.0272, -0.4034],
        [-1.5650, -1.0272, -0.4034],
        [-1.5650, -1.0272, -0.4034]])
tensor([-1.5650, -1.0272, -0.4034])
tensor([[-0.1358, -0.1905, -0.4474],
        [ 3.8544,  5.4059, 12.6960],
        [-0.0940, -0.1319, -0.3097]])
tensor([-0.1178,  3.3428, -0.0815])
tensor([[-11.6470, -13.3479,   6.3779]])
tensor([7.2874])


In [70]:
norm = 0
for m in net.parameters():
    norm += torch.sum(m.grad**2)
norm = torch.sqrt(norm)
if norm>1:
    for m in net.parameters():
        m.grad[:] *= 1/norm

for m in net.parameters():
    print(m.grad)

tensor([[-0.0617, -0.0405, -0.0159],
        [-0.0617, -0.0405, -0.0159],
        [-0.0617, -0.0405, -0.0159],
        [-0.0617, -0.0405, -0.0159]])
tensor([-0.0617, -0.0405, -0.0159])
tensor([[-0.0054, -0.0075, -0.0176],
        [ 0.1520,  0.2132,  0.5008],
        [-0.0037, -0.0052, -0.0122]])
tensor([-0.0046,  0.1319, -0.0032])
tensor([[-0.4594, -0.5265,  0.2516]])
tensor([0.2874])


In [ ]:
def grad_clipping(net, theta):
    """裁剪梯度"""
    if isinstance(net, nn.Module):
        params = [p for p in net.parameters() if p.requires_grad]
    else:
        params = net.params
    norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

In [56]:
nn.utils.clip_grad_norm_(net.parameters(), max_norm=1)
for m in net.parameters():
    print(m.grad)

tensor([[-0.0617, -0.0405, -0.0159],
        [-0.0617, -0.0405, -0.0159],
        [-0.0617, -0.0405, -0.0159],
        [-0.0617, -0.0405, -0.0159]])
tensor([-0.0617, -0.0405, -0.0159])
tensor([[-0.0054, -0.0075, -0.0176],
        [ 0.1520,  0.2132,  0.5008],
        [-0.0037, -0.0052, -0.0122]])
tensor([-0.0046,  0.1319, -0.0032])
tensor([[-0.4594, -0.5265,  0.2516]])
tensor([0.2874])


## [**梯度裁剪**]

对于长度为$T$的序列，我们在迭代中计算这$T$个时间步上的梯度，
将会在反向传播过程中产生长度为$\mathcal{O}(T)$的矩阵乘法链。
如 :numref:`sec_numerical_stability`所述，
当$T$较大时，它可能导致数值不稳定，
例如可能导致梯度爆炸或梯度消失。
因此，循环神经网络模型往往需要额外的方式来支持稳定训练。

一般来说，当解决优化问题时，我们对模型参数采用更新步骤。
假定在向量形式的$\mathbf{x}$中，
或者在小批量数据的负梯度$\mathbf{g}$方向上。
例如，使用$\eta > 0$作为学习率时，在一次迭代中，
我们将$\mathbf{x}$更新为$\mathbf{x} - \eta \mathbf{g}$。
如果我们进一步假设目标函数$f$表现良好，
即函数$f$在常数$L$下是*利普希茨连续的*（Lipschitz continuous）。
也就是说，对于任意$\mathbf{x}$和$\mathbf{y}$我们有：

$$|f(\mathbf{x}) - f(\mathbf{y})| \leq L \|\mathbf{x} - \mathbf{y}\|.$$

在这种情况下，我们可以安全地假设：
如果我们通过$\eta \mathbf{g}$更新参数向量，则

$$|f(\mathbf{x}) - f(\mathbf{x} - \eta\mathbf{g})| \leq L \eta\|\mathbf{g}\|,$$

这意味着我们不会观察到超过$L \eta \|\mathbf{g}\|$的变化。
这既是坏事也是好事。
坏的方面，它限制了取得进展的速度；
好的方面，它限制了事情变糟的程度，尤其当我们朝着错误的方向前进时。

有时梯度可能很大，从而优化算法可能无法收敛。
我们可以通过降低$\eta$的学习率来解决这个问题。
但是如果我们很少得到大的梯度呢？
在这种情况下，这种做法似乎毫无道理。
一个流行的替代方案是通过将梯度$\mathbf{g}$投影回给定半径
（例如$\theta$）的球来裁剪梯度$\mathbf{g}$。
如下式：

(**$$\mathbf{g} \leftarrow \min\left(1, \frac{\theta}{\|\mathbf{g}\|}\right) \mathbf{g}.$$**)

通过这样做，我们知道梯度范数永远不会超过$\theta$，
并且更新后的梯度完全与$\mathbf{g}$的原始方向对齐。
它还有一个值得拥有的副作用，
即限制任何给定的小批量数据（以及其中任何给定的样本）对参数向量的影响，
这赋予了模型一定程度的稳定性。
梯度裁剪提供了一个快速修复梯度爆炸的方法，
虽然它并不能完全解决问题，但它是众多有效的技术之一。

下面我们定义一个函数来裁剪模型的梯度，
模型是从零开始实现的模型或由高级API构建的模型。
我们在此计算了所有模型参数的梯度的范数。


In [47]:
def grad_clipping(net, theta):
    """裁剪梯度"""
    if isinstance(net, nn.Module):
        params = [p for p in net.parameters() if p.requires_grad]
    else:
        params = net.params
    norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

In [ ]:
params = [p for p in net.parameters() if p.requires_grad]
norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))

In [50]:
grad_clipping(net, theta=1)
for m in net.parameters():
    print(m.grad)

tensor([[-0.0617, -0.0405, -0.0159],
        [-0.0617, -0.0405, -0.0159],
        [-0.0617, -0.0405, -0.0159],
        [-0.0617, -0.0405, -0.0159]])
tensor([-0.0617, -0.0405, -0.0159])
tensor([[-0.0054, -0.0075, -0.0176],
        [ 0.1520,  0.2132,  0.5008],
        [-0.0037, -0.0052, -0.0122]])
tensor([-0.0046,  0.1319, -0.0032])
tensor([[-0.4594, -0.5265,  0.2516]])
tensor([0.2874])
